#### ML introduction part 2. Date: 2026-01-09

**Topics**:
> 1. LinearRegression/Logistics regression
> 2. RandomForest 

**Materials**:
> 1. [RandomForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

#### Логістична регресія/Logistics regression

Маємо об’єкт (рядок даних) з ознаками: x1, x2, x3, .... xn \
Модель рахує сирий скор: z = w(0) + w(1)x(1) ... w(n)(n) 

> w(0) - bias (зсув) \
> w(i) — ваги (коефіцієнти) \
> z - −∞,+∞

![](https://builtin.com/sites/www.builtin.com/files/styles/ckeditor_optimize/public/inline-images/sigmoid-activation-function-1_0.png)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


In [7]:
df = sns.load_dataset("titanic")
df.head()
df["sex"] = df["sex"].map({"male": 0, "female": 1})

In [8]:
features = ["sex", "age", "pclass", "fare"]
target = "survived"

df = df[features + [target]].dropna()


In [9]:
X = df[features]
y = df[target]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


In [11]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [12]:
model = LogisticRegression()
model.fit(X_train_scaled, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [13]:
proba = model.predict_proba(X_test_scaled)[:, 1]
pred = model.predict(X_test_scaled)


In [14]:
coef_df = pd.DataFrame({
    "feature": features,
    "coef": model.coef_[0]
})

coef_df


,feature,coef
0,sex,1.175361
1,age,-0.566312
2,pclass,-1.090164
3,fare,-0.024547


In [15]:
coef_df["odds_multiplier"] = np.exp(coef_df["coef"])
coef_df


,feature,coef,odds_multiplier
0,sex,1.175361,3.239313
1,age,-0.566312,0.567615
2,pclass,-1.090164,0.336161
3,fare,-0.024547,0.975752


In [22]:
passenger = pd.DataFrame({
    "sex": [0],
    "age": [46],
    "pclass": [1],
    "fare": [10]
})

passenger_scaled = scaler.transform(passenger)

probability = model.predict_proba(passenger_scaled)[0, 1]
probability


np.float64(0.3936001145506775)

Модель має не запам’ятовувати дані, а узагальнювати закономірності


##### Underfitting (недонавчання)
Модель занадто проста і не вловлює закономірності

**Ознаки:**
> погано працює і на train, і на test \
> низьки метрики 

**Опрацювати:**
> ускладнити модель \
> додати ознаки \
> прибрати занадто сильну регуляризацію


##### Overfitting (перенавчання)
Модель запам’ятала train, але не працює на нових даних

> train: дуже добре
> test: погано

**Приклад:** Decision Tree без обмежень модель, яка вивчила напам’ять шум


**Опрацювати:**
> обмежити складність \
> більше даних \
> регуляризація \
> крос-валідація

**Варіант 1**
> тільки sex \
> accuracy = 65% і на train, і на test


**Варіант 2**
> дерево без обмежень \
> train = 99%, test = 70%


**Варіант 3**
> train = 78%, test = 76%

Linear Regression/Лінійна регресія прогнозує **ЧИСЛО**

**Приклади:**
> ціна квартири \
> час доставки \
> дохід користувача

Модель будує пряму лінію:
y = w(0) + w(1)x(1) + w(2)(x2) + ....

> w(0) - базове значення \
> w(i) — наскільки зміниться результат, якщо ознака зросте на 1

Приклад:price = 1000 * area + 5000



In [23]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [24]:
data = fetch_california_housing(as_frame=True)
df = data.frame

df.head()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [25]:
target = "MedHouseVal" 
X = df[["MedInc"]] 
y = df[target]


In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42
)


In [27]:
model = LinearRegression()
model.fit(X_train, y_train)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [28]:
model.coef_, model.intercept_


(array([0.41788087]), np.float64(0.44967564199686105))

##### Ансамблі
Одна модель може помилятися, багато різних моделей — помиляються рідше

Random Forest - багато Decision Tree

**Кожне дерево:**
> навчається на випадковій підвибірці даних \
> використовує випадкову частину ознак

**Потім:**
класифікація - голосування \
регресія - середнє значення

| Decision Tree    | Random Forest       |
| ---------------- | ------------------- |
| легко overfit    | значно стабільніший |
| чутливий до шуму | згладжує шум        |
| 1 думка          | колективне рішення  |


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [30]:
df = sns.load_dataset("titanic")

features = ["sex", "age", "pclass", "fare"]
target = "survived"

df = df[features + [target]].dropna()
df["sex"] = df["sex"].map({"male": 1, "female": 0})

X = df[features]
y = df[target]


In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

In [32]:
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

rf.fit(X_train, y_train)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [33]:
pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))


Accuracy: 0.7932960893854749


In [34]:
importances = pd.DataFrame({
    "feature": features,
    "importance": rf.feature_importances_
}).sort_values("importance", ascending=False)

importances


,feature,importance
3,fare,0.322445
1,age,0.306446
0,sex,0.261975
2,pclass,0.109134


| Параметр           | Навіщо                      |
| ------------------ | --------------------------- |
| `n_estimators`     | кількість дерев             |
| `max_depth`        | обмеження складності        |
| `min_samples_leaf` | згладжування                |
| `max_features`     | скільки фіч дивиться дерево |


In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    "n_estimators": [100, 200, 250, 300],
    "max_depth": [None, 5, 10],
    "min_samples_leaf": [1, 3, 5, 7, 10]
}

grid = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

grid.fit(X_train, y_train)

grid.best_params_


NameError: name 'RandomForestClassifier' is not defined

In [36]:
best_rf = grid.best_estimator_

pred_best = best_rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred_best))

Accuracy: 0.8044692737430168
